In [187]:
#referred to lesson 9.05
#https://stackoverflow.com/questions/61757170/python-unstacked-dataframe-is-too-big-causing-int32-overflow
#https://stackoverflow.com/questions/28651079/pandas-unstack-problems-valueerror-index-contains-duplicate-entries-cannot-re#:~:text=The%20reason%20why%20you%20get%20ValueError%3A%20Index%20contains,%22%20date%20%22%20combinations%20are%20no%20longer%20unique.
#https://stackoverflow.com/questions/43945653/python-pandas-return-dataframe-where-value-count-is-above-a-set-number
#https://stackoverflow.com/questions/54822879/how-can-i-combine-multiple-sparse-and-dense-matrices-together
import pandas as pd
import numpy as np
from scipy import sparse
from sklearn.metrics.pairwise import pairwise_distances, cosine_distances, cosine_similarity

In [2]:
df=pd.read_csv('./data/video_games.csv')

In [3]:
df.isnull().sum()

customer_id           0
review_id             0
product_id            0
product_parent        0
product_title         0
star_rating           0
helpful_votes         0
total_votes           0
verified_purchase     0
review_date           0
full_review          47
dtype: int64

In [4]:
df.dropna(inplace=True)

In [120]:
df['review_date'] = pd.to_datetime(df['review_date'])

In [6]:
df.head()

,customer_id,review_id,product_id,product_parent,product_title,star_rating,helpful_votes,total_votes,verified_purchase,review_date,full_review
0,12039526,RTIS3L2M1F5SM,B001CXYMFS,737716809,Thrustmaster T-Flight Hotas X Flight Stick,5,0,0,1,2015-08-31,amazing joystick I especially love twist Used ...
1,2331478,R3BH071QLH8QMC,B0029CSOD2,98937668,Hidden Mysteries: Titanic Secrets of the Fatef...,1,0,1,1,2015-08-31,One Star poor quality work advertised
2,52495923,R127K9NTSXA2YH,B00GOOSV98,23143350,GelTabz Performance Thumb Grips - PlayStation ...,3,0,0,1,2015-08-31,good could bettee nice tend slip away stick in...
3,14533949,R32ZWUXDJPW27Q,B00Y074JOM,821342511,Zero Suit Samus amiibo - Japan Import (Super S...,4,0,0,1,2015-08-31,Great flawed Great amiibo great collecting Qua...
4,17521011,R2F0POU5K6F73F,B008XHCLFO,24234603,Protection for your 3DS XL,5,0,0,1,2015-08-31,A Must I 2012 2013 XL durable comfortable real...


In [9]:
df.columns

Index(['customer_id', 'review_id', 'product_id', 'product_parent',
       'product_title', 'star_rating', 'helpful_votes', 'total_votes',
       'verified_purchase', 'review_date', 'full_review'],
      dtype='object')

I need to see if any customers wrote multiple reviews for the same product. That would throw an error in the pivot table.

In [50]:
df.groupby('customer_id')['product_id'].value_counts().sort_values(ascending=False)

customer_id  product_id
38142327     B00005NH6B    13
43411792     B00000JRSB    13
50763007     B0000AHOOJ    10
25214010     B0000C4M22    10
30995260     B00005BW6Z    10
                           ..
38010719     B000035Y6N     1
38010773     B001C91H4G     1
             B001DL8PES     1
38010873     B0083RDT8C     1
10018        B006VE40JQ     1
Name: product_id, Length: 1641949, dtype: int64

Hm... for example, customer 38142327 wrote 13 reviews for product B00005NH6B... what do those reviews look like?

In [31]:
df.loc[(df2['product_id']=='B00005NH6B') & (df2['customer_id']==38142327)]

,customer_id,review_id,product_id,product_parent,product_title,star_rating,helpful_votes,total_votes,verified_purchase,review_date,full_review
1571527,38142327,R3NY8KHMR5X2KP,B00005NH6B,199086987,Batman Vengeance,1,1,2,0,2002-06-29,Pitiful I huge Batman fan Anything Batman bet ...
1572890,38142327,R2U1NNLJS8OHHQ,B00005NH6B,199086987,Batman Vengeance,5,0,1,0,2002-06-17,Batman come alooong way From 1930 new milleniu...
1572901,38142327,R122RU8UP19R4T,B00005NH6B,199086987,Batman Vengeance,5,0,0,0,2002-06-17,FUN FOR FAMILY If like playing game family spe...
1572912,38142327,R22Z951P3JMRBQ,B00005NH6B,199086987,Batman Vengeance,5,0,0,0,2002-06-17,Best game hit store There isnt superhero like ...
1572927,38142327,R2RCRESG64DH7D,B00005NH6B,199086987,Batman Vengeance,5,1,1,0,2002-06-17,Fairly good game kid 12 First I would like say...
1572929,38142327,RHMREFYJF0IWP,B00005NH6B,199086987,Batman Vengeance,4,0,2,0,2002-06-17,WHATCHA LOOKIN AT WAAAAAAAAAAAAAAAAAAAAAAAAAAA...
1572933,38142327,R1TZYG1TID6GE6,B00005NH6B,199086987,Batman Vengeance,4,1,2,0,2002-06-17,ha spider man beat I really dont feel like wri...
1572935,38142327,R1QDQSKQ7XJPLI,B00005NH6B,199086987,Batman Vengeance,5,0,1,0,2002-06-17,You sorry miss UBI Soft best Good work Warner ...
1572936,38142327,R2RI3Y4DAWBISL,B00005NH6B,199086987,Batman Vengeance,5,1,2,0,2002-06-17,Best yet If PS2 nice fun game get It pass time...
1572979,38142327,RVES7ZUE7F43B,B00005NH6B,199086987,Batman Vengeance,4,0,0,0,2002-06-17,One DC Comic greatest superheroes finally go P...


So this person made 13 reviews for the same item, rating 4 or 5 stars EXCEPT for the most recent one (6/29) which was 1 star. Perhaps the ending was disappointing?

In [79]:
review_bools = df.groupby('customer_id')['product_id'].value_counts()>1
#T/F if more than 1 review for same product

In [87]:
review_bools[review_bools==True].count()

5309

Okay, based on this there are 5,309 customers with at least 2 reviews for the same product.  

I could try to get the most recent review for each customer based on date. It's likely reasonable to assume that their latest review is probably the one that best reflects their final sentiment about the product. Unfortunately, because I don't have precise timestamps, if they wrote two reviews on the same day, I can't know which was their last word on the product. Other options would be to average all of their reviews' star values or to simply drop them, as they're a fairly small proportion of the overall set.

In [117]:
xtra_rev_cs = []
for key, value in dict(review_bools[review_bools==True]).items():
    xtra_rev_cs.append({key[0]:key[1]}) #return customer id and product id ONLY
#list of customer id numbers for those with more than 1 review for same item
xtra_rev_cs[:5]  #preview

[{198273: 'B00DNGQQUQ'},
 {224068: 'B0049LAVB4'},
 {815919: 'B0000TSR4C'},
 {884129: 'B000WMEEB2'},
 {884129: 'B00ERDGMT4'}]

In [140]:
df[(df['customer_id'] == 38142327) & 
   (df['product_id']=='B00005NH6B')].sort_values(
    by='review_date', ascending=False).index[1:] #test

Int64Index([1572890, 1572901, 1572912, 1572927, 1572929, 1572933, 1572935,
            1572936, 1572979, 1572980, 1572983, 1573111],
           dtype='int64')

Okay - with the above code in conjunction with the xtra_rev_cs dictionary, I can filter the dataframe by customer_id and product_id to isolate the rows that pertain to a single product reviewed mutliple times by the same user, and then order it by date where the most recent date will be on top. While this does open the possibility that two reviews with the same date may be conflated as far as which ACTUALLY came first, the examples I've previewed in this situation are typically duplicates (i.e. someone submitted the same review twice). In that circumstance, either review is equally valid to keep.  

The next step is to loop through my xtra_rev_cs dictionary and create a list of all the indexes EXCEPT the most recent one. I'll drop those others, and be left with one review per person per item.

In [134]:
rev_indexes_to_drop = []
for pair in xtra_rev_cs:
    for key, value in pair.items():
        rev_indexes_to_drop.append(  #add index numbers to the empty list
        df[(df['customer_id'] == key) &    #where customer id is the key from xtra_rev_cs...
       (df['product_id']==value)].sort_values( #and product id is the value from xtra_rev_cs
        by='review_date', ascending=False).index[1:] #starting with the SECOND index number
        )
rev_indexes_to_drop[:5] #preview

[Int64Index([26651], dtype='int64'),
 Int64Index([70501], dtype='int64'),
 Int64Index([90766], dtype='int64'),
 Int64Index([422385], dtype='int64'),
 Int64Index([421883], dtype='int64')]

In [144]:
rev_indexes_to_drop[0][0] #test

26651

In [145]:
ritd_2 = []
for n in rev_indexes_to_drop:
    for k in n:
        ritd_2.append(k)
ritd_2[:5] #preview

[26651, 70501, 90766, 422385, 421883]

In [146]:
len(ritd_2) #6140 rows of duplicate reviews to be dropped

6140

In [147]:
df.drop(index=ritd_2, inplace=True) #drop all index numbers in list ritd_2
df.groupby('customer_id')['product_id'].value_counts().sort_values(ascending=False) #check if gone

customer_id  product_id
53096565     B00006599W    1
17664004     B000U5RRX8    1
17663554     B000SH3XGS    1
17663609     B002TK1PX0    1
17663621     B00E20STAW    1
                          ..
38073024     B0088MVOES    1
38073066     B006VB2UNM    1
38073092     B007P6Y684    1
             B00A9ZHWH0    1
10018        B006VE40JQ    1
Name: product_id, Length: 1641949, dtype: int64

In [148]:
df2 = df[['customer_id', 'product_id', 'product_title', 'star_rating']].copy()

In [150]:
#pivot = pd.pivot_table(df2, index='product_title', columns='customer_id', values='star_rating')

My first attempt at making a pivot table from this data failed because the created dataframe is too big. I'll have to do it in chunks.

In [172]:
len(df2)/37

44377.0

In [168]:
df2[:10_000] #partial dataframe

,customer_id,product_id,product_title,star_rating
0,12039526,B001CXYMFS,Thrustmaster T-Flight Hotas X Flight Stick,5
1,2331478,B0029CSOD2,Hidden Mysteries: Titanic Secrets of the Fatef...,1
2,52495923,B00GOOSV98,GelTabz Performance Thumb Grips - PlayStation ...,3
3,14533949,B00Y074JOM,Zero Suit Samus amiibo - Japan Import (Super S...,4
4,17521011,B008XHCLFO,Protection for your 3DS XL,5
...,...,...,...,...
9996,2597447,B00005LVWL,Nancy Drew: Treasure In The Royal Tower - PC,5
9997,34800440,B00004WFVZ,Microsoft Combat Flight Simulator 2: Pacific ...,5
9998,13984861,B00VETEZ34,PlayStation 4 500GB Console - Batman Arkham Kn...,5
9999,21531468,B00KAED850,Far Cry 4,5


In [170]:
pivot = pd.pivot_table(df2[:100_000], index='product_title', 
                       columns='customer_id', 
                       values='star_rating') #test

In [184]:
def chunky_pivot(dataframe):
    final_pivot = pd.Series(dtype='str')
    size = len(dataframe)
    new_size = size
    chunk_size=50_000
    while new_size > 50_000:
        pivot = pd.pivot_table(df2[(chunk_size-50_000):chunk_size], index='product_title', 
                       columns='customer_id', 
                       values='star_rating')
        sparse_pivot = sparse.csr_matrix(pivot.fillna(0))
        final_pivot = pd.concat([final_pivot, sparse_pivot]) #aggregate sparse matrices
        new_size-=50_000 #keep track of how much of df is left to process
        chunk_size+=50_000 #next set of rows
    #last pivot with remaining rows
    pivot = pd.pivot_table(df2[size-new_size:],
                           index='product_title', 
                       columns='customer_id', 
                       values='star_rating')
    sparse_pivot = sparse.csr_matrix(pivot.fillna(0))
    final_pivot = pd.concat([final_pivot, sparse_pivot]) #aggregate sparse matrices 
    return pd.concat([matrix for matrix in final_pivot]) #combine all sparse matrices into one

In [185]:
vg_sparse = chunky_pivot(df2)

TypeError: cannot concatenate object of type '<class 'scipy.sparse.csr.csr_matrix'>'; only Series and DataFrame objs are valid

In [188]:
chunk_sz = 10_000
chunks = [n for n in range(0, len(df2), chunk_sz)]

In [189]:
ch_df = pd.concat(
    [df2.loc[chunks[i]:chunks[i+1]-1].pivot(
        index='product_title', columns='customer_id', values='star_rating')
    for i in range(0,len(chunks)-1)])

ValueError: Index contains duplicate entries, cannot reshape